<a href="https://colab.research.google.com/github/EduardoAve/Data-science-portfolio/blob/main/models/SEM_Austria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structural Equation Modeling (SEM) for Labour Well-being

This notebook implements a path analysis model to investigate the impact of various working conditions on burnout, mediated by motivation and psychological vulnerability.

## 1. Installation of Necessary Libraries

If you do not have `semopy` installed, the following cell will install it. `semopy` is the main library for performing SEM in Python. `statsmodels` is used for calculating the Variance Inflation Factor (VIF).

In [1]:
!pip install semopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 2.9 MB/s eta 0:00:00
  Created wheel for semopy: filename=semopy-2.3.11-py3-none-any.whl size=1659682 sha256=2a831cbf80c33cd6db1c876cc028a793031d72d65071c0ed543fff18caa3152b
  Stored in directory: /root/.cache/pip/wheels/d2/9a/31/fae291ff6a649bad125037eef8c7cc63d8c542e14bdcccea37
Successfully built semopy


## 2. Importing Libraries

We import the Python libraries that will be used for data analysis, numerical calculations, SEM modeling, and statistical tests.

In [2]:
import pandas as pd
import numpy as np
import semopy
import scipy.stats # For Sobel test
from statsmodels.stats.outliers_influence import variance_inflation_factor # For VIF
from statsmodels.tools.tools import add_constant # For VIF
import time # For timing
# import seaborn as sns # Uncomment if you want to generate correlation heatmaps
# import matplotlib.pyplot as plt # Uncomment if you want to generate correlation heatmaps

## 3. Data Loading and Preparation

We load the dataset from the GitHub URL. Then, we rename some columns to ensure compatibility with `semopy` and create composite variables for the two types of motivation (controlled and autonomous) by averaging their respective indicators.

In [3]:
# Step 0: Load data from URL and rename columns
data_url = "https://raw.githubusercontent.com/EduardoAve/Labour-well-being/refs/heads/main/data/02_processed/Final_Dataset_Processed.csv"
df = None
try:
    df = pd.read_csv(data_url)
    print(f"CSV file loaded successfully from: {data_url}")
except Exception as e:
    print(f"An error occurred while loading the CSV from the URL: {e}")
    exit()

try:
    df.rename(columns={'Effort [%]': 'Effort_perc', 'Income EURO': 'Income_EURO'}, inplace=True)
    print("Columns 'Effort [%]' and 'Income EURO' renamed to 'Effort_perc' and 'Income_EURO' (if they existed).")
except KeyError as e:
    print(f"Warning during renaming: {e}. A column to be renamed was not present.")
print("-" * 50)

# Step 1: Create composite motivation variables
motivation_cols_for_controlled = ['WM_Extrinsic_Social', 'WM_Extrinsic_Material', 'WM_Introjected_Motivation']
motivation_cols_for_autonomous = ['WM_Identified_Motivation', 'WM_Intrinsic_Motivation']

missing_motivation_cols = [col for col in motivation_cols_for_controlled + motivation_cols_for_autonomous if col not in df.columns]
if missing_motivation_cols:
    print(f"Error: The following motivation indicator columns are missing from the DataFrame: {missing_motivation_cols}")
    exit()

df['WM_Controlled_Motivation'] = df[motivation_cols_for_controlled].mean(axis=1)
df['WM_Autonomous_Motivation'] = df[motivation_cols_for_autonomous].mean(axis=1)
print("Composite motivation variables (WM_Controlled_Motivation, WM_Autonomous_Motivation) created.")
print("-" * 50)

# Asumiendo que '2' es Chequia y '2' es Académico (¡VERIFICA ESTO!)
df_filtered = df[(df['Country'] == 1) & (df['Academic/Non-academic'] == 2)].copy()

# Verificar el resultado del filtro
print(f"Número de filas original: {df.shape[0]}")
print(f"Número de filas después de filtrar: {df_filtered.shape[0]}")

# --- ¡AQUÍ ESTÁ EL TRUCO! ---
# Sobrescribimos el df original con el df ya filtrado.
df = df_filtered

print(f"\nEl DataFrame 'df' ha sido actualizado. Ahora contiene las {df.shape[0]} filas de la muestra filtrada.")
print("No se necesitan más cambios en el resto del código.")
print("-" * 50)
# ==================================================================

CSV file loaded successfully from: https://raw.githubusercontent.com/EduardoAve/Labour-well-being/refs/heads/main/data/02_processed/Final_Dataset_Processed.csv
Columns 'Effort [%]' and 'Income EURO' renamed to 'Effort_perc' and 'Income_EURO' (if they existed).
--------------------------------------------------
Composite motivation variables (WM_Controlled_Motivation, WM_Autonomous_Motivation) created.
--------------------------------------------------
Número de filas original: 1949
Número de filas después de filtrar: 637

El DataFrame 'df' ha sido actualizado. Ahora contiene las 637 filas de la muestra filtrada.
No se necesitan más cambios en el resto del código.
--------------------------------------------------


## 4. Data Diagnostics and Collinearity Analysis (VIF)

We define the list of 9 working condition indicators that will be used as observed predictors. We perform a NaN value check on all columns that will enter the SEM model. Subsequently, we calculate the Variance Inflation Factor (VIF) for the 9 working condition indicators to assess multicollinearity.

In [4]:
# List of observed predictors for 'Working Conditions'
indicadores_X_observados = [
    'Avg_Work_Hours_HE', 'Income_EURO', 'Effort_perc', 'Policy_Influence',
    'Academic_Resources', 'Performance_Pressure', 'Perceived_Autonomy',
    'Quality_Leadership', 'Sense_Community'
]

# Initial Data Diagnostics
columnas_del_modelo_actual = [
    'WM_Controlled_Motivation', 'WM_Autonomous_Motivation',
    'Vulnerability', 'Burnout_Score'
] + indicadores_X_observados

missing_cols_in_df = [col for col in columnas_del_modelo_actual if col not in df.columns]
if missing_cols_in_df:
    print(f"CRITICAL Error: Necessary columns for the model are missing from the DataFrame: {missing_cols_in_df}")
    exit()

print("Checking for NaNs in the final model columns:")
nan_counts = df[columnas_del_modelo_actual].isnull().sum()
print(nan_counts)
total_nans_in_model_cols = nan_counts.sum()
print(f"Total NaNs in these model columns: {total_nans_in_model_cols}")
if total_nans_in_model_cols > 0:
    print("WARNING: Missing data detected. FIML will be attempted for SEM fitting.")
print("-" * 50)

# Step 1.A: Collinearity Analysis (VIF) for the 9 Working Condition indicators
print("Calculating Variance Inflation Factor (VIF) for the 9 Working Condition indicators...")
X_predictores_df_vif = df[indicadores_X_observados].copy()
X_predictores_df_vif.dropna(inplace=True) # VIF calculation requires no NaNs

if len(X_predictores_df_vif) > len(indicadores_X_observados) and X_predictores_df_vif.shape[1] > 1:
    X_with_const_vif = add_constant(X_predictores_df_vif, prepend=False)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_predictores_df_vif.columns
    vif_data["VIF"] = [variance_inflation_factor(X_with_const_vif.values, i) for i in range(X_predictores_df_vif.shape[1])]
    print("\nVariance Inflation Factor (VIF):")
    print(vif_data.sort_values('VIF', ascending=False))
else:
    print("Not enough data or predictors to calculate VIF after handling NaNs (if any)." )
print("VIF Interpretation Guide:")
print("  VIF = 1: No collinearity.")
print("  VIF between 1 and 5: Moderate collinearity (generally acceptable).")
print("  VIF > 5 or 10: High collinearity, may be problematic.")
print("-" * 50)

Checking for NaNs in the final model columns:
WM_Controlled_Motivation    0
WM_Autonomous_Motivation    0
Vulnerability               0
Burnout_Score               0
Avg_Work_Hours_HE           0
Income_EURO                 0
Effort_perc                 0
Policy_Influence            0
Academic_Resources          0
Performance_Pressure        0
Perceived_Autonomy          0
Quality_Leadership          0
Sense_Community             0
dtype: int64
Total NaNs in these model columns: 0
--------------------------------------------------
Calculating Variance Inflation Factor (VIF) for the 9 Working Condition indicators...

Variance Inflation Factor (VIF):
                feature       VIF
0     Avg_Work_Hours_HE  1.872021
1           Income_EURO  1.871812
6    Perceived_Autonomy  1.569229
7    Quality_Leadership  1.450830
8       Sense_Community  1.334798
4    Academic_Resources  1.316463
3      Policy_Influence  1.162444
2           Effort_perc  1.125249
5  Performance_Pressure  1.093939
VIF

## 5. Path Analysis Model Specification

We define the structure of the path analysis model. Each of the 9 working condition indicators acts as an individual observed predictor for the motivation variables and for burnout. Motivations predict vulnerability, and vulnerability (along with working conditions) predicts burnout.

In [5]:
# Step 2: Path Analysis Model Specification
predictores_X_str = " + ".join(indicadores_X_observados)
model_spec_path_analysis = f"""
# Structural Model with X indicators as separate observed predictors
WM_Controlled_Motivation ~ {predictores_X_str}
WM_Autonomous_Motivation ~ {predictores_X_str}
Vulnerability ~ WM_Controlled_Motivation + WM_Autonomous_Motivation
Burnout_Score ~ {predictores_X_str} + Vulnerability
"""
print("Path Analysis Model Specification:")
print(model_spec_path_analysis)
print("-" * 50)

Path Analysis Model Specification:

# Structural Model with X indicators as separate observed predictors
WM_Controlled_Motivation ~ Avg_Work_Hours_HE + Income_EURO + Effort_perc + Policy_Influence + Academic_Resources + Performance_Pressure + Perceived_Autonomy + Quality_Leadership + Sense_Community
WM_Autonomous_Motivation ~ Avg_Work_Hours_HE + Income_EURO + Effort_perc + Policy_Influence + Academic_Resources + Performance_Pressure + Perceived_Autonomy + Quality_Leadership + Sense_Community
Vulnerability ~ WM_Controlled_Motivation + WM_Autonomous_Motivation
Burnout_Score ~ Avg_Work_Hours_HE + Income_EURO + Effort_perc + Policy_Influence + Academic_Resources + Performance_Pressure + Perceived_Autonomy + Quality_Leadership + Sense_Community + Vulnerability

--------------------------------------------------


## 6. SEM Model Fitting

We create an instance of the `semopy` model with the defined specification and fit the model to the data. The MLW (Maximum Likelihood Wishart) estimator is used as there is no missing data in the model variables. If NaNs were present, FIML would be attempted.

In [6]:
# Step 3: SEM Model Creation and Fitting
model = semopy.Model(model_spec_path_analysis)
results_optimization = None
converged_based_on_solver = False

print("Attempting to fit the path analysis model...")
estimator_to_use = 'MLW'
df_for_model = df[columnas_del_modelo_actual].copy() # Use a copy of the relevant columns

if total_nans_in_model_cols > 0:
    print(f"WARNING: {total_nans_in_model_cols} NaNs detected. Attempting to use FIML.")
    estimator_to_use = 'FIML'
    # For FIML, pass the DataFrame with NaNs (df_for_model in this case)
else:
    print(f"No NaNs detected in model columns. Using estimator {estimator_to_use}.")
    # For MLW, ensure df_for_model has no NaNs (it shouldn't at this point based on prior check)
    df_for_model.dropna(inplace=True)

try:
    results_optimization = model.fit(data=df_for_model, obj=estimator_to_use)
    if hasattr(results_optimization, 'success') and results_optimization.success:
        converged_based_on_solver = True
        print("  Optimizer interpretation: SUCCESS reported.")
    elif hasattr(results_optimization, 'status') and results_optimization.status == 0:
        converged_based_on_solver = True
        print("  Optimizer interpretation: STATUS CODE 0 reported (typically success).")
    else:
        print("  Optimizer interpretation: Success NOT explicitly reported or known success status code not found.")
    print("-" * 50)
except Exception as e:
    print(f"CRITICAL Error during model fitting with {estimator_to_use}: {e}")
    print("-" * 50)

Attempting to fit the path analysis model...
No NaNs detected in model columns. Using estimator MLW.
  Optimizer interpretation: SUCCESS reported.
--------------------------------------------------


## 7. Model Fit Evaluation

If the model converged successfully, we calculate and print the goodness-of-fit indices (CFI, TLI, RMSEA, etc.) to assess how well the model represents the data.

In [7]:
# Step 4: Model Fit Evaluation
if converged_based_on_solver:
    print("Attempting to calculate model fit indices...")
    try:
        fit_indices = semopy.calc_stats(model)
        print("\nModel Fit Indices:")
        print(fit_indices.T)
    except Exception as e:
        print(f"Error calculating fit indices: {e}")
    print("-" * 50)
else:
    print("Model optimization did not report success or failed. Fit indices will not be calculated.")
    print("-" * 50)

Attempting to calculate model fit indices...

Model Fit Indices:
                     Value
DoF              57.000000
DoF Baseline     87.000000
chi2             87.604237
chi2 p-value      0.005682
chi2 Baseline  1672.198381
CFI               0.980694
GFI               0.947611
AGFI              0.920038
NFI               0.947611
TLI               0.970533
RMSEA             0.029055
AIC              67.724947
BIC             219.255116
LogLik            0.137526
--------------------------------------------------


## 8. Interpretation of Direct Path Coefficients

If the model converged, we inspect the estimated parameters. Both non-standardized and standardized estimates are printed. Standardized estimates are particularly useful for comparing the relative strength of the effects of different predictors.

In [8]:
# Step 5: Interpretation of Direct Path Coefficients
estimates_df = None
estimates_df_std = None
if converged_based_on_solver:
    print("Attempting to inspect parameters...")
    try:
        estimates_df = model.inspect() # Non-standardized
        estimates_df_std = model.inspect(std_est=True) # Standardized

        if estimates_df is not None and not estimates_df.empty:
            print("\nModel Parameter Estimates (Non-Standardized):")
            print(estimates_df)
            if estimates_df_std is not None and not estimates_df_std.empty:
                print("\nModel Parameter Estimates (STANDARDIZED):")
                print(estimates_df_std)
            else:
                print("\nCould not calculate standardized estimates.")

            # Check for negative residual variances for endogenous variables
            if 'Std. Err' in estimates_df.columns: # Check if Std. Err column exists, implying successful parameter estimation
                 residual_variances = estimates_df[estimates_df['op'] == '~~']
                 self_variances = residual_variances[residual_variances['lval'] == residual_variances['rval']]
                 endogenous_obs_vars = ['WM_Controlled_Motivation', 'WM_Autonomous_Motivation', 'Vulnerability', 'Burnout_Score']
                 problematic_variances = self_variances[self_variances['lval'].isin(endogenous_obs_vars) & (self_variances['Estimate'] < 0)]
                 if not problematic_variances.empty:
                     print("\nWARNING: Negative residual variances detected for endogenous variables (Heywood cases):")
                     print(problematic_variances)
        else:
            print("\nWARNING: model.inspect() for non-standardized estimates returned empty or None.")
            estimates_df = None
            estimates_df_std = None
        print("-" * 50)
    except Exception as e:
        print(f"Error inspecting model results: {e}")
        estimates_df = None
        estimates_df_std = None
        print("-" * 50)
else:
    print("Model optimization did not report success or failed. Parameters will not be displayed.")
    print("-" * 50)

Attempting to inspect parameters...

Model Parameter Estimates (Non-Standardized):
                        lval  op                      rval  Estimate  \
0   WM_Controlled_Motivation   ~         Avg_Work_Hours_HE -0.001063   
1   WM_Controlled_Motivation   ~               Income_EURO -0.000063   
2   WM_Controlled_Motivation   ~               Effort_perc -0.000248   
3   WM_Controlled_Motivation   ~          Policy_Influence -0.131134   
4   WM_Controlled_Motivation   ~        Academic_Resources  0.059020   
5   WM_Controlled_Motivation   ~      Performance_Pressure  0.216990   
6   WM_Controlled_Motivation   ~        Perceived_Autonomy -0.041240   
7   WM_Controlled_Motivation   ~        Quality_Leadership  0.020370   
8   WM_Controlled_Motivation   ~           Sense_Community  0.059001   
9   WM_Autonomous_Motivation   ~         Avg_Work_Hours_HE  0.013397   
10  WM_Autonomous_Motivation   ~               Income_EURO -0.000005   
11  WM_Autonomous_Motivation   ~               Effort

#Boostrapping for Indirect Effects (Mediation)

In [9]:
# Step 7: Bootstrapping for Indirect Effects (Mediation)

if converged_based_on_solver and estimates_df is not None and not estimates_df.empty:
    print("\nInitiating Bootstrapping for Indirect Effects (with enhanced debugging)...")

    # Number of bootstrap samples
    # For actual research, 2000-5000 is recommended.
    # For debugging, start with a small number like 20-50.
    n_bootstrap_samples = 2000 # << ADJUST AS NEEDED (e.g., to 1000 or 5000 for final run)

    bootstrapped_indirect_effects = {}
    mediation_paths_definitions = []

    if 'indicadores_X_observados' not in globals():
        # This is a fallback, ensure 'indicadores_X_observados' is correctly defined from previous cells
        indicadores_X_observados = [
            'Avg_Work_Hours_HE', 'Income_EURO', 'Effort_perc', 'Policy_Influence',
            'Academic_Resources', 'Performance_Pressure', 'Perceived_Autonomy',
            'Quality_Leadership', 'Sense_Community'
        ]

    for indicador_x in indicadores_X_observados:
        mediation_paths_definitions.append({
            "name": f"{indicador_x} -> WM_C -> V -> BO",
            "a_path_lval": "WM_Controlled_Motivation", "a_path_rval": indicador_x,
            "b_path_lval": "Vulnerability", "b_path_rval": "WM_Controlled_Motivation",
            "c_path_lval": "Burnout_Score", "c_path_rval": "Vulnerability"
        })
        mediation_paths_definitions.append({
            "name": f"{indicador_x} -> WM_A -> V -> BO",
            "a_path_lval": "WM_Autonomous_Motivation", "a_path_rval": indicador_x,
            "b_path_lval": "Vulnerability", "b_path_rval": "WM_Autonomous_Motivation",
            "c_path_lval": "Burnout_Score", "c_path_rval": "Vulnerability"
        })

    for path_def in mediation_paths_definitions:
        bootstrapped_indirect_effects[path_def["name"]] = []

    start_time = time.time()
    successful_fits_and_inspects = 0
    fit_failures = 0
    inspect_failures = 0
    path_component_failures_count = 0

    first_path_find_failure_logged = False

    # Ensure df_for_model, model_spec_path_analysis, and estimator_to_use are defined from previous cells
    # These should be available if you are running the notebook cells sequentially.
    if 'df_for_model' not in globals() or 'model_spec_path_analysis' not in globals() or 'estimator_to_use' not in globals():
        print("Error: df_for_model, model_spec_path_analysis, or estimator_to_use is not defined. Please ensure previous cells defining these have been run.")
        # As a safety, you could add fallback definitions here if necessary for standalone testing,
        # but it's better to ensure the notebook flow defines them.
        # For example:
        # df_for_model = df[columnas_del_modelo_actual].copy()
        # if total_nans_in_model_cols == 0 and estimator_to_use == 'MLW': df_for_model.dropna(inplace=True)
        # model_spec_path_analysis = f"""...""" # Your full model spec string
        # estimator_to_use = 'MLW' # Or 'FIML' based on your NaN check
        # exit() # Or handle this error more gracefully

    print(f"Starting bootstrap loop with {n_bootstrap_samples} samples...")
    for i in range(n_bootstrap_samples):
        if (i + 1) % (n_bootstrap_samples // 10 if n_bootstrap_samples >=10 else 1) == 0 or n_bootstrap_samples < 10 or i==0:
            print(f"  Processing bootstrap sample {i + 1}/{n_bootstrap_samples}...")

        df_bootstrap = df_for_model.sample(n=len(df_for_model), replace=True, random_state=i)
        model_bootstrap = semopy.Model(model_spec_path_analysis)

        opt_result_bootstrap = None
        estimates_bootstrap = None
        current_sample_fit_successful = False
        current_sample_inspect_successful = False

        try:
            # ***** CORRECTED FIT CALL: Removed solver_options *****
            opt_result_bootstrap = model_bootstrap.fit(data=df_bootstrap, obj=estimator_to_use)

            if hasattr(opt_result_bootstrap, 'success') and opt_result_bootstrap.success:
                current_sample_fit_successful = True
            elif hasattr(opt_result_bootstrap, 'status') and opt_result_bootstrap.status == 0:
                current_sample_fit_successful = True

            if current_sample_fit_successful:
                try:
                    estimates_bootstrap = model_bootstrap.inspect()
                    if estimates_bootstrap is None or estimates_bootstrap.empty:
                        print(f"    DEBUG: Sample {i+1}: inspect() returned None or empty despite fit success.")
                        inspect_failures += 1
                    else:
                        current_sample_inspect_successful = True
                except Exception as e_inspect:
                    print(f"    DEBUG: Sample {i+1} inspect() FAILED after successful fit: {e_inspect}")
                    inspect_failures += 1
            else:
                print(f"    DEBUG: Sample {i+1} fit FAILED or did not report success. Optimizer message: {getattr(opt_result_bootstrap, 'message', 'N/A')}")
                fit_failures += 1
        except Exception as e_fit:
            print(f"    DEBUG: Sample {i+1} CRITICAL FIT ERROR: {e_fit}")
            fit_failures += 1

        if current_sample_fit_successful and current_sample_inspect_successful:
            successful_fits_and_inspects +=1
            all_paths_found_for_this_sample_ie = True
            for path_def in mediation_paths_definitions:
                a_coeff, b_coeff, c_coeff = np.nan, np.nan, np.nan
                a_row_empty, b_row_empty, c_row_empty = True, True, True
                try:
                    a_row = estimates_bootstrap.loc[(estimates_bootstrap['lval'] == path_def["a_path_lval"]) & (estimates_bootstrap['rval'] == path_def["a_path_rval"])]
                    b_row = estimates_bootstrap.loc[(estimates_bootstrap['lval'] == path_def["b_path_lval"]) & (estimates_bootstrap['rval'] == path_def["b_path_rval"])]
                    c_row = estimates_bootstrap.loc[(estimates_bootstrap['lval'] == path_def["c_path_lval"]) & (estimates_bootstrap['rval'] == path_def["c_path_rval"])]

                    a_row_empty = a_row.empty
                    b_row_empty = b_row.empty
                    c_row_empty = c_row.empty

                    if not a_row_empty and not b_row_empty and not c_row_empty:
                        a_coeff = a_row['Estimate'].iloc[0]
                        b_coeff = b_row['Estimate'].iloc[0]
                        c_coeff = c_row['Estimate'].iloc[0]
                        indirect_effect_boot = a_coeff * b_coeff * c_coeff
                        bootstrapped_indirect_effects[path_def["name"]].append(indirect_effect_boot)
                    else:
                        bootstrapped_indirect_effects[path_def["name"]].append(np.nan)
                        all_paths_found_for_this_sample_ie = False
                        if not first_path_find_failure_logged:
                            print(f"\n    DEBUG PATH FIND FAILURE: Sample {i+1}, Path: {path_def['name']}")
                            print(f"      a_row empty: {a_row_empty} (lval='{path_def['a_path_lval']}', rval='{path_def['a_path_rval']}')")
                            print(f"      b_row empty: {b_row_empty} (lval='{path_def['b_path_lval']}', rval='{path_def['b_path_rval']}')")
                            print(f"      c_row empty: {c_row_empty} (lval='{path_def['c_path_lval']}', rval='{path_def['c_path_rval']}')")
                            if estimates_bootstrap is not None:
                                print(f"      Content of estimates_bootstrap for sample {i+1} (first 15 rows):")
                                print(estimates_bootstrap.head(15).to_string())
                            else:
                                print(f"      estimates_bootstrap is None for sample {i+1}.")
                            first_path_find_failure_logged = True
                except (KeyError, IndexError) as e_path:
                    print(f"    DEBUG PATH KEY/INDEX ERROR: Sample {i+1}, Path: {path_def['name']}, Error: {e_path}")
                    bootstrapped_indirect_effects[path_def["name"]].append(np.nan)
                    all_paths_found_for_this_sample_ie = False

            if not all_paths_found_for_this_sample_ie:
                 path_component_failures_count +=1
        else:
            for path_def in mediation_paths_definitions:
                bootstrapped_indirect_effects[path_def["name"]].append(np.nan)

    end_time = time.time()
    print(f"\nBootstrapping completed in {end_time - start_time:.2f} seconds.")
    print(f"Total bootstrap samples attempted: {n_bootstrap_samples}")
    print(f"  Successfully fitted & inspected samples: {successful_fits_and_inspects}")
    print(f"  Fit failures (or no success reported by optimizer): {fit_failures}")
    print(f"  Inspect failures (after successful fit): {inspect_failures}")
    print(f"  Samples where not all IE path components were found (after fit & inspect OK): {path_component_failures_count}")
    print("-" * 50)

    bootstrap_summary_list = []
    print("\nBootstrap Results for Indirect Effects (95% Percentile CIs):")
    for path_name, ie_values in bootstrapped_indirect_effects.items():
        valid_ie_values = [val for val in ie_values if not np.isnan(val)]
        min_samples_for_ci = max(int(0.1 * n_bootstrap_samples), 20)

        if len(valid_ie_values) >= min_samples_for_ci:
            mean_ie = np.mean(valid_ie_values)
            lower_ci = np.percentile(valid_ie_values, 2.5)
            upper_ci = np.percentile(valid_ie_values, 97.5)
            significant = not (lower_ci < 0 < upper_ci)

            print(f"  Path: {path_name}")
            print(f"    Mean Indirect Effect: {mean_ie:.4f}")
            print(f"    95% CI: [{lower_ci:.4f}, {upper_ci:.4f}]")
            print(f"    Significant (p < .05 based on CI): {'Yes' if significant else 'No'}")
            bootstrap_summary_list.append([path_name, mean_ie, lower_ci, upper_ci, 'Yes' if significant else 'No', len(valid_ie_values)])
        else:
            print(f"  Path: {path_name}")
            print(f"    Not enough valid bootstrap samples to calculate CI (got {len(valid_ie_values)} out of {n_bootstrap_samples} attempted, need at least {min_samples_for_ci}).")
            bootstrap_summary_list.append([path_name, np.nan, np.nan, np.nan, f"N/A (need {min_samples_for_ci})", len(valid_ie_values)])

    df_bootstrap_summary = pd.DataFrame(bootstrap_summary_list,
                                        columns=['Mediation_Path', 'Mean_Indirect_Effect',
                                                 'CI_Lower_2.5%', 'CI_Upper_97.5%', 'Significant_p<.05', 'Num_Valid_Samples'])
    print("\nSummary Table of Bootstrapped Indirect Effects:")
    print(df_bootstrap_summary)
    print("-" * 50)

else:
    print("Main model did not converge or parameters are unavailable. Bootstrapping for indirect effects cannot be performed.")


Initiating Bootstrapping for Indirect Effects (with enhanced debugging)...
Starting bootstrap loop with 2000 samples...
  Processing bootstrap sample 1/2000...
  Processing bootstrap sample 200/2000...
  Processing bootstrap sample 400/2000...
  Processing bootstrap sample 600/2000...
  Processing bootstrap sample 800/2000...
  Processing bootstrap sample 1000/2000...
  Processing bootstrap sample 1200/2000...
  Processing bootstrap sample 1400/2000...
  Processing bootstrap sample 1600/2000...
  Processing bootstrap sample 1800/2000...
  Processing bootstrap sample 2000/2000...

Bootstrapping completed in 138.21 seconds.
Total bootstrap samples attempted: 2000
  Successfully fitted & inspected samples: 2000
  Fit failures (or no success reported by optimizer): 0
  Inspect failures (after successful fit): 0
  Samples where not all IE path components were found (after fit & inspect OK): 0
--------------------------------------------------

Bootstrap Results for Indirect Effects (95% Pe

## 9. Calculation and Significance Testing of Indirect Effects (Mediation)

We systematically calculate the indirect effects for the 18 main mediation pathways (9 working conditions x 2 motivation types → Vulnerability → Burnout). The Sobel test is used to assess the statistical significance of each indirect effect. The results are presented individually and in a summary table.

In [10]:
# Step 6: Calculation and Significance Testing of Indirect Effects (Mediation)
if estimates_df is not None and not estimates_df.empty and 'Std. Err' in estimates_df.columns:
    print("\nCalculating All Relevant Indirect Effects (Sobel Test)...")
    try:
        # Common paths for mediators M1->M2 and M2->Y
        path_WM_C_V = estimates_df.loc[(estimates_df['lval'] == 'Vulnerability') & (estimates_df['rval'] == 'WM_Controlled_Motivation')]
        path_WM_A_V = estimates_df.loc[(estimates_df['lval'] == 'Vulnerability') & (estimates_df['rval'] == 'WM_Autonomous_Motivation')]
        path_V_BO = estimates_df.loc[(estimates_df['lval'] == 'Burnout_Score') & (estimates_df['rval'] == 'Vulnerability')]

        # Check if all base paths for mediators were found and have standard errors
        if path_WM_C_V.empty or path_WM_A_V.empty or path_V_BO.empty or \
           path_WM_C_V['Std. Err'].isnull().any() or path_WM_A_V['Std. Err'].isnull().any() or \
           path_V_BO['Std. Err'].isnull().any():
            print("Error: Common mediator paths (M->V or V->BO) or their Std. Err are missing. Cannot calculate indirect effects.")
        else:
            b1_C, se_b1_C = path_WM_C_V['Estimate'].iloc[0], path_WM_C_V['Std. Err'].iloc[0]
            b1_A, se_b1_A = path_WM_A_V['Estimate'].iloc[0], path_WM_A_V['Std. Err'].iloc[0]
            c1, se_c1 = path_V_BO['Estimate'].iloc[0], path_V_BO['Std. Err'].iloc[0]

            resultados_mediacion_lista = [] # To store mediation results

            for indicador_x in indicadores_X_observados:
                print(f"\n--- Mediation for: {indicador_x} ---")

                # PATH 1: indicador_x -> WM_Controlled_Motivation (a1) -> Vulnerability (b1_C) -> Burnout_Score (c1)
                path_a1_C = estimates_df.loc[
                    (estimates_df['lval'] == 'WM_Controlled_Motivation') &
                    (estimates_df['rval'] == indicador_x)
                ]
                if not path_a1_C.empty and not path_a1_C['Std. Err'].isnull().any():
                    a1, se_a1 = path_a1_C['Estimate'].iloc[0], path_a1_C['Std. Err'].iloc[0]
                    IE_C = a1 * b1_C * c1
                    # Sobel test for SE of indirect effect (a*b*c)
                    SE_IE_C_sq = (a1**2 * b1_C**2 * se_c1**2) + \
                                 (a1**2 * c1**2 * se_b1_C**2) + \
                                 (b1_C**2 * c1**2 * se_a1**2)
                    p_IE_C_val, z_IE_C_val, SE_IE_C_val = "N/A", "N/A", "N/A"
                    if SE_IE_C_sq > 0 and not np.isnan(SE_IE_C_sq) and se_a1 > 0 and se_b1_C > 0 and se_c1 > 0:
                        SE_IE_C = np.sqrt(SE_IE_C_sq)
                        SE_IE_C_val = f"{SE_IE_C:.4f}"
                        if SE_IE_C > 1e-9: # Avoid division by zero or very small SE
                            z_IE_C = IE_C / SE_IE_C
                            p_IE_C = 2 * (1 - scipy.stats.norm.cdf(abs(z_IE_C))) # Two-tailed test
                            p_IE_C_val = f"{p_IE_C:.4f}"
                            z_IE_C_val = f"{z_IE_C:.2f}"
                    print(f"  Indirect Effect ({indicador_x} -> WM_C -> V -> BO): {IE_C:.4f}, SE: {SE_IE_C_val}, z: {z_IE_C_val}, p: {p_IE_C_val}")
                    resultados_mediacion_lista.append([f"{indicador_x} -> WM_C -> V -> BO", IE_C, SE_IE_C_val, z_IE_C_val, p_IE_C_val])
                else:
                    print(f"    Path or Std.Err missing for: {indicador_x} -> WM_Controlled_Motivation")
                    resultados_mediacion_lista.append([f"{indicador_x} -> WM_C -> V -> BO", np.nan, "N/A", "N/A", "N/A"])

                # PATH 2: indicador_x -> WM_Autonomous_Motivation (a1_aut) -> Vulnerability (b1_A) -> Burnout_Score (c1)
                path_a1_A = estimates_df.loc[
                    (estimates_df['lval'] == 'WM_Autonomous_Motivation') &
                    (estimates_df['rval'] == indicador_x)
                ]
                if not path_a1_A.empty and not path_a1_A['Std. Err'].isnull().any():
                    a1_aut, se_a1_aut = path_a1_A['Estimate'].iloc[0], path_a1_A['Std. Err'].iloc[0]
                    IE_A = a1_aut * b1_A * c1
                    SE_IE_A_sq = (a1_aut**2 * b1_A**2 * se_c1**2) + \
                                 (a1_aut**2 * c1**2 * se_b1_A**2) + \
                                 (b1_A**2 * c1**2 * se_a1_aut**2)
                    p_IE_A_val, z_IE_A_val, SE_IE_A_val = "N/A", "N/A", "N/A"
                    if SE_IE_A_sq > 0 and not np.isnan(SE_IE_A_sq) and se_a1_aut > 0 and se_b1_A > 0 and se_c1 > 0:
                        SE_IE_A = np.sqrt(SE_IE_A_sq)
                        SE_IE_A_val = f"{SE_IE_A:.4f}"
                        if SE_IE_A > 1e-9:
                            z_IE_A = IE_A / SE_IE_A
                            p_IE_A = 2 * (1 - scipy.stats.norm.cdf(abs(z_IE_A)))
                            p_IE_A_val = f"{p_IE_A:.4f}"
                            z_IE_A_val = f"{z_IE_A:.2f}"
                    print(f"  Indirect Effect ({indicador_x} -> WM_A -> V -> BO): {IE_A:.4f}, SE: {SE_IE_A_val}, z: {z_IE_A_val}, p: {p_IE_A_val}")
                    resultados_mediacion_lista.append([f"{indicador_x} -> WM_A -> V -> BO", IE_A, SE_IE_A_val, z_IE_A_val, p_IE_A_val])
                else:
                    print(f"    Path or Std.Err missing for: {indicador_x} -> WM_Autonomous_Motivation")
                    resultados_mediacion_lista.append([f"{indicador_x} -> WM_A -> V -> BO", np.nan, "N/A", "N/A", "N/A"])

            # Convert mediation results list to a Pandas DataFrame for pretty printing
            df_resultados_mediacion = pd.DataFrame(resultados_mediacion_lista,
                                                 columns=['Mediation_Path', 'Indirect_Effect', 'SE_Sobel', 'z_value', 'p_value'])
            print("\nSummary Table of Calculated Indirect Effects:")
            print(df_resultados_mediacion)
        print("-" * 50)
    except KeyError as e:
        print(f"Error (KeyError) during indirect effect calculation: {e}. Check variable names in 'estimates_df'.")
    except Exception as e:
        print(f"Error during Sobel test calculation for indirect effects: {e}")
elif converged_based_on_solver and (estimates_df is None or estimates_df.empty or 'Std. Err' not in estimates_df.columns):
    print("Parameter estimates or standard errors are unavailable. Cannot calculate indirect effects.")
else:
    print("Model did not converge adequately or estimates are unavailable. Cannot calculate indirect effects.")

print("\nSEM (Path Analysis) complete.")


Calculating All Relevant Indirect Effects (Sobel Test)...

--- Mediation for: Avg_Work_Hours_HE ---
  Indirect Effect (Avg_Work_Hours_HE -> WM_C -> V -> BO): -0.0000, SE: 0.0002, z: -0.24, p: 0.8106
  Indirect Effect (Avg_Work_Hours_HE -> WM_A -> V -> BO): -0.0008, SE: 0.0003, z: -2.83, p: 0.0047

--- Mediation for: Income_EURO ---
  Indirect Effect (Income_EURO -> WM_C -> V -> BO): -0.0000, SE: 0.0000, z: -1.61, p: 0.1076
  Indirect Effect (Income_EURO -> WM_A -> V -> BO): 0.0000, SE: 0.0000, z: 0.16, p: 0.8695

--- Mediation for: Effort_perc ---
  Indirect Effect (Effort_perc -> WM_C -> V -> BO): -0.0000, SE: 0.0000, z: -0.38, p: 0.7066
  Indirect Effect (Effort_perc -> WM_A -> V -> BO): -0.0001, SE: 0.0000, z: -1.33, p: 0.1822

--- Mediation for: Policy_Influence ---
  Indirect Effect (Policy_Influence -> WM_C -> V -> BO): -0.0055, SE: 0.0022, z: -2.52, p: 0.0119
  Indirect Effect (Policy_Influence -> WM_A -> V -> BO): -0.0006, SE: 0.0025, z: -0.26, p: 0.7981

--- Mediation for: Ac

# 10. Analysis Conclusion

This notebook has successfully implemented and evaluated a path analysis model to explore the complex relationships between specific working conditions, motivation (controlled and autonomous), psychological vulnerability, and burnout among the surveyed labour force. The key findings from this analytical process are summarized below:

A. Data Quality and Model Suitability:

No Missing Data: The final set of variables used in the model (WM_Controlled_Motivation, WM_Autonomous_Motivation, Vulnerability, Burnout_Score, and the 9 specific working condition indicators) had no missing values, allowing for a complete case analysis.

Low Collinearity: The Variance Inflation Factor (VIF) analysis for the 9 working condition indicators (acting as simultaneous predictors) revealed very low VIF values for all variables (all < 1.6). This indicates that multicollinearity among these predictors is not a concern, and the individual effects of each working condition can be reliably estimated.

Good Model Fit: The specified path analysis model demonstrated a good fit to the observed data. Key fit indices include:

CFI = 0.953

TLI = 0.928

RMSEA = 0.0396
(While the χ
2
  p-value was 0.000, this is common in large samples (N=992) and does not negate the good fit indicated by other robust indices).

B. Key Findings from Direct Effects (Based on Standardized Coefficients):

The model allowed for the examination of direct effects of working conditions on motivation and burnout, as well as the effects of motivation on vulnerability, and vulnerability on burnout.

Influences on Motivation:

Controlled Motivation (WM_Controlled_Motivation) was most strongly and positively predicted by Performance_Pressure (β=0.189). It was most strongly and negatively predicted by Perceived_Autonomy (β=−0.060) and Policy_Influence (β=−0.084). Other significant, albeit smaller, predictors included Academic_Resources (+), Income_EURO (-), Sense_Community (+), Avg_Work_Hours_HE (+), and Quality_Leadership (+).

Autonomous Motivation (WM_Autonomous_Motivation) was most strongly and positively predicted by Perceived_Autonomy (β=0.364). Other significant positive predictors included Performance_Pressure (β=0.104), Avg_Work_Hours_HE (β=0.094), Effort_perc (β=0.068), Policy_Influence (β=0.075), and Sense_Community (β=0.070). Academic_Resources showed a significant negative effect (β=−0.024, p=0.031).

Influences on Vulnerability (Vulnerability):

WM_Controlled_Motivation significantly increased vulnerability (β=0.158).

WM_Autonomous_Motivation significantly decreased vulnerability (β=−0.287), showing a stronger relative impact than controlled motivation.

Influences on Burnout (Burnout_Score):

Vulnerability was a strong positive predictor of burnout (β=0.231).

Among the direct effects of working conditions, Perceived_Autonomy was the strongest negative predictor (protective factor) of burnout (β=−0.176).

Performance_Pressure was the strongest positive predictor (risk factor) of burnout (β=0.176).

Other significant direct predictors of burnout included Avg_Work_Hours_HE (+), Effort_perc (+), Policy_Influence (+), Academic_Resources (-), Quality_Leadership (-), and Sense_Community (-).

C. Key Findings from Indirect Effects (Mediation Analysis):

The analysis of indirect effects, using the Sobel test, revealed several significant mediation pathways, indicating that working conditions influence burnout not only directly but also through the proposed chain of motivation and vulnerability.

Strongest Mediation Pathways:

Perceived_Autonomy demonstrated the most substantial significant negative indirect effect on Burnout_Score via the WM_Autonomous_Motivation → Vulnerability chain (Indirect Effect ≈ -0.0224, p < 0.0001). This suggests that higher autonomy reduces burnout 위험 by fostering autonomous motivation, which in turn reduces vulnerability.

Performance_Pressure showed significant indirect effects on Burnout_Score through both motivational pathways:

Via WM_Controlled_Motivation → Vulnerability: Positive indirect effect (increases burnout, p < 0.0001).

Via WM_Autonomous_Motivation → Vulnerability: Negative indirect effect (decreases burnout, p < 0.0001). This is an interesting finding, suggesting that while performance pressure directly increases burnout and also increases it via controlled motivation, it simultaneously has a burnout-reducing indirect effect through the autonomous motivation pathway.

Other Notable Significant Mediations:

Avg_Work_Hours_HE had a significant negative indirect effect on burnout via WM_Autonomous_Motivation and Vulnerability (p=0.0003).

Income_EURO had a significant negative indirect effect via WM_Controlled_Motivation and Vulnerability (p=0.0264).

Effort_perc had a significant negative indirect effect via WM_Autonomous_Motivation and Vulnerability (p=0.0026).

Policy_Influence had significant negative indirect effects via both WM_Controlled_Motivation (p=0.0017) and WM_Autonomous_Motivation (p=0.0013) pathways.

Academic_Resources had a significant positive indirect effect via WM_Controlled_Motivation and Vulnerability (p=0.0108).

Sense_Community had a significant negative indirect effect via WM_Autonomous_Motivation and Vulnerability (p=0.0046).

D. Overall Conclusion:

The implemented path analysis model provides a good fit to the data and offers valuable insights into the mechanisms linking specific working conditions to burnout. The findings underscore the distinct roles of controlled and autonomous motivation as mediators, with autonomous motivation and perceived autonomy emerging as particularly crucial for mitigating vulnerability and burnout. Several working conditions exert their influence on burnout both directly and indirectly through these motivational and vulnerability pathways. These results highlight specific areas for potential intervention to enhance well-being in the workplace. Further research could explore these relationships in different contexts or with longitudinal data to infer causality more strongly.